In [33]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [34]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

# Scratch notebook for handling long stop to stop segments

* goal is to provide additional map detail for rural/express routes

In [35]:
## KART speedmap as displayed on site
itp_id = 148
analysis_date = dt.date(2022, 6, 1)

In [36]:
pbar = tqdm()

0it [00:00, ?it/s]

In [37]:
kart_analysis = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
less than 1km of data
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less t

In [6]:
# gdf = kart_analysis.routelines >> filter(_.shape_id == '3')
# shared_utils.rt_utils.map_line(gdf)

In [42]:
kart_analysis.rt_trips >> filter(_.route_short_name == '2')

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
167,148,0,2022-06-01,-4138729168852673726,126,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,18:24:28.500000,Northbound,53.323435,Kings Area Rural Transit
170,148,0,2022-06-01,6584290056450939487,109,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,09:56:51,Northbound,187.597633,Kings Area Rural Transit
171,148,0,2022-06-01,-3362092941811335345,120,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,15:27:06,Northbound,87.662068,Kings Area Rural Transit
172,148,0,2022-06-01,-426090013608928590,115,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,12:58:51,Northbound,62.434907,Kings Area Rural Transit
174,148,0,2022-06-01,-1425587692006046116,107,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,08:56:51,Northbound,53.445856,Kings Area Rural Transit
175,148,0,2022-06-01,5403644744137733838,104,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,07:26:36,Northbound,53.623522,Kings Area Rural Transit
179,148,0,2022-06-01,8187097971622236901,123,2,0,49,2021-07-02,2099-01-01,2,Orange Line,None,3,16:55:37,Northbound,93.792242,Kings Area Rural Transit


In [8]:
# kart_analysis.stop_delay_view = kart_analysis.stop_delay_view >> mutate(last_loc = _.shape_meters.shift(1))

In [39]:
# long_trip = kart_analysis.stop_delay_view >> filter(_.trip_id == '947') >> arrange(_.stop_sequence)

In [10]:
# test_center = (long_trip >> arrange(_.shape_meters)).iloc[10:49, :]

In [45]:
really_fast_bus = kart_analysis.position_interpolators['104']['rt']

In [46]:
dir(really_fast_bus)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attach_shape',
 '_dt_array',
 '_linear_reference',
 '_position_cleaning_count',
 '_shape_array',
 '_shift_calculate',
 'calitp_itp_id',
 'cleaned_positions',
 'debug_dict',
 'detailed_map_view',
 'detailed_speed_map',
 'direction',
 'direction_id',
 'entity_id',
 'export_detailled_map',
 'mean_speed_mph',
 'median_time',
 'position_gdf',
 'position_type',
 'progressing_positions',
 'route_id',
 'route_short_name',
 'service_date',
 'shape',
 'shape_id',
 'time_at_position',
 'time_col',
 'time_of_day',
 'total_meters',
 'total_seconds',
 'trip_id',
 'trip_key',
 'vehicle_id']

In [60]:
df = (
    tbl.views.gtfs_schedule_dim_shapes()
    >> filter(
        _.calitp_extracted_at <= analysis_date,
        _.calitp_deleted_at > analysis_date,
    )
    >> filter(_.calitp_itp_id == 148, _.shape_id == '49')
    # >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
    >> collect()
)

In [63]:
df = df >> arrange(_.shape_pt_sequence)

In [64]:
import geopandas as gpd
import folium

In [65]:
df >> head(3)

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at,calitp_hash,shape_key,calitp_deleted_at
17,148,0,49,36.325647,-119.652688,1,None,2021-04-15,ru0ixCJbt8tkUUqzC4Aqwg==,-4907651418736819562,2099-01-01
52,148,0,49,36.326609,-119.652883,2,None,2021-04-15,/P1ceA4gkYRAiN3Bz/Cosg==,-8708430766074839456,2099-01-01
42,148,0,49,36.326440,-119.653930,3,None,2021-04-15,GHPi+x3rcs/0x3WY7Cv8Pg==,-6331366513554695594,2099-01-01


In [66]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.shape_pt_lon, df.shape_pt_lat), crs='EPSG:4326')

In [86]:
gdf = gdf >> select(-_.calitp_extracted_at, -_.calitp_deleted_at)

In [72]:
folium.Map?

Init signature:
folium.Map(
    location=None,
    width='100%',
    height='100%',
    left='0%',
    top='0%',
    position='relative',
    tiles='OpenStreetMap',
    attr=None,
    min_zoom=0,
    max_zoom=18,
    zoom_start=10,
    min_lat=-90,
    max_lat=90,
    min_lon=-180,
    max_lon=180,
    max_bounds=False,
    crs='EPSG3857',
    control_scale=False,
    prefer_canvas=False,
    no_touch=False,
    disable_3d=False,
    png_enabled=False,
    zoom_control=True,
    **kwargs,
)
Docstring:     
Create a Map with Folium and Leaflet.js

Generate a base map of given width and height with either default
tilesets or a custom tileset URL. The following tilesets are built-in
to Folium. Pass any of the following to the "tiles" keyword:

    - "OpenStreetMap"
    - "Mapbox Bright" (Limited levels of zoom for free tiles)
    - "Mapbox Control Room" (Limited levels of zoom for free tiles)
    - "Stamen" (Terrain, Toner, and Watercolor)
    - "Cloudmade" (Must pass API key)
    - "Mapb

In [87]:
m = folium.Map(location=[36.32, -119.65], tiles='cartodbpositron')

In [91]:
import branca

In [95]:
colormap = branca.colormap.LinearColormap(['#ffffed', '#202a44'], vmin=gdf.shape_pt_sequence.min(),
                                        vmax=gdf.shape_pt_sequence.max())

In [97]:
colormap(14)

'#e9eaddff'

In [98]:
# branca.colormap.LinearColormap?

In [101]:
folium.GeoJson(gdf >> select(_.shape_pt_sequence,_.geometry),
               style_function=lambda x: {
            "fillColor": colormap(x["properties"]['shape_pt_sequence'])
            # if x["properties"][plot_col] is not None
            # else "gray",
            # "color": "#FFFFFF",
            # "fillOpacity": 0.8,
            # "weight": 0.2,
            },
              ).add_to(m)

In [102]:
m

In [54]:
shared_utils.rt_utils.map_line(kart_analysis.routelines >> filter(_.shape_id == '49'))

In [47]:
really_fast_bus.__class__

rt_analysis.VehiclePositionsInterpolator

In [51]:
really_fast_bus.position_gdf

,route_long_name,route_desc,route_type,header_timestamp,vehicle_timestamp,vehicle_longitude,vehicle_latitude,geometry,shape_meters,secs_from_last,meters_from_last,progressed,speed_from_last
0,Orange Line,None,3,2022-06-01 07:09:30,2022-06-01 07:09:21,-119.652695,36.325752,POINT (31139.609 -187862.306),11.616606,NaN,NaN,True,NaN
1,Orange Line,None,3,2022-06-01 07:10:00,2022-06-01 07:09:51,-119.652710,36.326252,POINT (31138.061 -187806.748),10059.788182,30.0,10048.171575,True,334.939053
2,Orange Line,None,3,2022-06-01 07:11:30,2022-06-01 07:11:21,-119.652730,36.326263,POINT (31136.263 -187805.532),10061.282578,90.0,1.494397,True,0.016604
3,Orange Line,None,3,2022-06-01 07:13:30,2022-06-01 07:13:21,-119.652725,36.326275,POINT (31136.707 -187804.197),10062.526840,120.0,1.244262,True,0.010369
4,Orange Line,None,3,2022-06-01 07:15:30,2022-06-01 07:15:22,-119.652725,36.326260,POINT (31136.713 -187805.864),10060.881558,121.0,-1.645282,False,-0.013597
5,Orange Line,None,3,2022-06-01 07:16:30,2022-06-01 07:16:21,-119.652800,36.326420,POINT (31129.923 -187788.108),10079.510697,59.0,18.629140,True,0.315748
6,Orange Line,None,3,2022-06-01 07:17:00,2022-06-01 07:16:51,-119.653915,36.326110,POINT (31030.078 -187822.923),240.282847,30.0,-9839.227850,False,-327.974262
7,Orange Line,None,3,2022-06-01 07:17:30,2022-06-01 07:17:21,-119.653420,36.325375,POINT (31074.759 -187904.438),344.740583,30.0,104.457737,True,3.481925
8,Orange Line,None,3,2022-06-01 07:18:00,2022-06-01 07:17:51,-119.651880,36.325623,POINT (31212.735 -187876.373),485.567025,30.0,140.826442,True,4.694215
9,Orange Line,None,3,2022-06-01 07:18:30,2022-06-01 07:18:06,-119.650470,36.325863,POINT (31339.058 -187849.238),9781.575224,15.0,9296.008199,True,619.733880


In [49]:
len(really_fast_bus.position_gdf) - len(really_fast_bus.cleaned_positions)

45

In [50]:
really_fast_bus.cleaned_positions

,route_long_name,route_desc,route_type,header_timestamp,vehicle_timestamp,vehicle_longitude,vehicle_latitude,geometry,shape_meters,secs_from_last,meters_from_last,progressed,speed_from_last
9961,Orange Line,None,3,2022-06-01 07:09:30,2022-06-01 07:09:21,-119.652695,36.325752,POINT (31139.609 -187862.306),11.616606,NaN,NaN,True,NaN
9962,Orange Line,None,3,2022-06-01 07:10:00,2022-06-01 07:09:51,-119.652710,36.326252,POINT (31138.061 -187806.748),10059.788182,30.0,10048.171575,True,334.939053
9963,Orange Line,None,3,2022-06-01 07:11:30,2022-06-01 07:11:21,-119.652730,36.326263,POINT (31136.263 -187805.532),10061.282578,90.0,1.494397,True,0.016604
9964,Orange Line,None,3,2022-06-01 07:13:30,2022-06-01 07:13:21,-119.652725,36.326275,POINT (31136.707 -187804.197),10062.526840,120.0,1.244262,True,0.010369
9966,Orange Line,None,3,2022-06-01 07:16:30,2022-06-01 07:16:21,-119.652800,36.326420,POINT (31129.923 -187788.108),10079.510697,180.0,16.983857,True,0.094355


In [43]:
kart_analysis.position_interpolators['104']['rt'].detailed_speed_map()

speed above 80 for trip 5403644744137733838, dropping


/opt/conda/lib/python3.10/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:110: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [14]:
kart_analysis.export_views_gcs()

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

In [15]:
import rt_filter_map_plot

In [16]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [17]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

found parquet


In [28]:
rt_day.set_filter(start_time='06:00', end_time='08:00', route_names=['2'])

In [29]:
filtered = rt_day._filter(rt_day.stop_delay_view) >> arrange(_.stop_sequence)

In [30]:
m = rt_day.segment_speed_map()

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:285: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this op

In [31]:
m

In [32]:
filtered >> arrange(_.stop_sequence)

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id,actual_time,delay_seconds
19,10042.676769,100,KART Transfer Center,POINT (31143.590 -187823.175),49,5.403645e+18,104,1.0,2022-06-01 07:09:50.948912,2,2,0,2022-06-01 07:09:50.948912,0.0
0,1118.130143,454,7th & Douty,POINT (31834.368 -187759.051),49,5.403645e+18,104,2.0,2022-06-01 07:09:24.303627,2,2,0,2022-06-01 07:09:24.303627,0.0
1,1370.135583,455,7th & Brown,POINT (32081.831 -187711.447),49,5.403645e+18,104,3.0,2022-06-01 07:09:25.056018,2,2,0,2022-06-01 07:09:25.056018,0.0
2,1689.808976,456,7th & East,POINT (32395.496 -187649.915),49,5.403645e+18,104,4.0,2022-06-01 07:09:26.010441,2,2,0,2022-06-01 07:09:26.010441,0.0
3,2157.201020,76,10th @ Carwash,POINT (32552.444 -187310.266),49,5.403645e+18,104,5.0,2022-06-01 07:09:27.405895,2,2,0,2022-06-01 07:09:27.405895,0.0
4,2379.566937,273,10th & Myrtle,POINT (32556.883 -187088.215),49,5.403645e+18,104,6.0,2022-06-01 07:09:28.069795,2,2,0,2022-06-01 07:09:28.069795,0.0
5,2728.687449,274,10th @ Fraternal Hall,POINT (32559.041 -186738.821),49,5.403645e+18,104,7.0,2022-06-01 07:09:29.112135,2,2,0,2022-06-01 07:09:29.112135,0.0
6,3107.425221,276,10th & Malone,POINT (32555.798 -186360.109),49,5.403645e+18,104,8.0,2022-06-01 07:09:30.242901,2,2,0,2022-06-01 07:09:30.242901,0.0
7,3549.471230,184,10th & Grangeville,POINT (32553.387 -185918.051),49,5.403645e+18,104,9.0,2022-06-01 07:09:31.562682,2,2,0,2022-06-01 07:09:31.562682,0.0
8,4338.983776,457,10th & Leland Way,POINT (32544.987 -185128.959),49,5.403645e+18,104,10.0,2022-06-01 07:09:33.919864,2,2,0,2022-06-01 07:09:33.919864,0.0
